# Mask-Filling task

## Import libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
import random

In [3]:
import tqdm

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

c:\Users\Giulia\AppData\Local\Programs\anaconda3\envs\venv_fe\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import string

In [6]:
import os

In [7]:
os.getcwd()

'c:\\Users\\Giulia\\Documents\\MyGithub\\Français_Exercises\\french-exercises\\notebooks'

## Dataset

In [8]:
df = pd.DataFrame(np.zeros((1,1)))
with open('../data/input/lepetitprinceexupery.txt', 'r') as f:
     lines = f.readlines()
     for line in lines:
            data = line.strip().split("\t")
            for el in data:
                phrases = el.strip().split(".")
                for phrase in phrases:
                    df = pd.DataFrame(np.append(df.values, [[phrase]], axis=0))

In [9]:
list_punctuation_to_remove = "!#$%&()*+-./:;<=>?@[\]^_{|}~)"

In [10]:
def check_row_is_to_remove(row):
    # convert and remove punctuation
    new_row = row.translate(str.maketrans('','',list_punctuation_to_remove))
    # we want it has at least 5 words.
    len_row = len(new_row.split())
    if len_row < 5:
        to_clean = True
    else:
        to_clean = False
    return [to_clean, new_row]

In [11]:
df.rename(columns = {0:'original'}, inplace = True)

In [12]:
df[['check','new']] = [[check_row_is_to_remove(el)[0],check_row_is_to_remove(el)[1]] for el in df['original']]

In [13]:
df = df.loc[df['check']== False]

In [14]:
df.head(5)

,original,check,new
9,"Lorsque j'avais six ans j'ai vu, une fois, une...",False,"Lorsque j'avais six ans j'ai vu, une fois, une..."
10,Ça représentait un serpent boa qui avalait un...,False,Ça représentait un serpent boa qui avalait un...
11,Voilà la copie du dessin,False,Voilà la copie du dessin
14,"On disait dans le livre :""Les serpents boas av...",False,"On disait dans le livre ""Les serpents boas ava..."
15,Ensuite ils ne peuvent plus bouger et ils dor...,False,Ensuite ils ne peuvent plus bouger et ils dor...


In [15]:
df['original'].iloc[0]

'Lorsque j\'avais six ans j\'ai vu, une fois, une magnifique image, dans un livre sur la Forêt Vierge qui s\'appelait"Histoires Vécues"'

## Model

In [16]:
tokenizer = AutoTokenizer.from_pretrained("moussaKam/mbarthez")

model = AutoModelForSeq2SeqLM.from_pretrained("moussaKam/mbarthez")

Example of BART model for mask-filling task in NLP.

A french sequence to sequence pretrained model based on BART.
BARThez is pretrained by learning to reconstruct a corrupted input sentence. A corpus of 66GB of french raw text is used to carry out the pretraining.
Unlike already existing BERT-based French language models such as CamemBERT and FlauBERT, BARThez is particularly well-suited for generative tasks (such as abstractive summarization), since not only its encoder but also its decoder is pretrained.

## Decode the input

In [17]:
def f_decode_input(txt):
    input_ids = tokenizer([txt], return_tensors="pt")["input_ids"]
    logits = model(input_ids).logits
    masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
    probs = logits[0, masked_index].softmax(dim=0)
    values, predictions = probs.topk(5)
    return tokenizer.decode(predictions).split()

## Example

In [33]:
TXT = "Lorsque j'avais six ans j'ai vu, une fois, une <mask> image."

In [34]:
TXT2 = "J'ai montré mon chef-d'oeuvre aux grandes <mask> et je leur ai demandé si mon dessin leur faisait peur."

In [35]:
TXT3 = "je faisais l'expérience sur <mask> de mon dessin numéro Un"

In [36]:
for txt in [TXT, TXT2, TXT3]:
    print(f_decode_input(txt))

['image', 'fois', 'seule', 'autre', 'photo']
['personnes', 'écoles', 'et', 'heures']
['la', 'le', 'l', 'de', 'les']


## Training

In [19]:
dict_QA = {'question' : [], 'answer' : []}

for row in df.iterrows():
    words = row[1]['new'].split()
    rnd = random.randint(0,len(words)-1)
    to_mask = words[rnd]
    while len(to_mask)<2:
        rnd = random.randint(0,len(words)-1)
        to_mask = words[rnd]    
    words[rnd] = '<mask>'
    to_txt = ''
    for el in words:
        to_txt += el + ' '
    txt = to_txt[:-1]
    predictions = f_decode_input(txt)
    if to_mask not in predictions:
        predictions = predictions[:-1]
        predictions.append(to_mask)
    dict_QA['question'].append(txt)
    dict_QA['answer'].append(predictions)

## End

In [57]:
df['question'] = dict_QA['question']

<ipython-input-57-a3918db5873b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['question'] = dict_QA['question']


In [61]:
df['answer'] = dict_QA['answer'].astype()

<ipython-input-61-7305735fe1af>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'] = dict_QA['answer']


## Save the output

In [63]:
df.drop(labels = ['original','check'], axis = 1 , inplace = True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [66]:
df.to_csv("output/df_QA.csv", index = False)